In [ ]:
import ipyparallel as ipp
import os
os.environ["XDG_RUNTIME_DIR"] = "/tmp"
nprocs = 3
rc = ipp.Cluster(engine_launcher_class="mpi", n=nprocs).start_and_connect_sync()

In [ ]:
%%px
import gmsh
import dolfinx as df
import numpy as np
from mpi4py import MPI
comm_world = MPI.COMM_WORLD

In [ ]:
%%px

# Create a gmsh mesh of a unit square with various regions marked as physical groups
gmsh.initialize()

gmsh.model.add("square")
lc = 3e-2
gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
gmsh.model.geo.addPoint(1, 0, 0, lc, 2)
gmsh.model.geo.addPoint(1, 0.4, 0, lc, 3)
gmsh.model.geo.addPoint(1, 0.8, 0, lc, 4)
gmsh.model.geo.addPoint(1, 1, 0, lc, 5)
gmsh.model.geo.addPoint(0.5, 1, 0, lc, 6)
gmsh.model.geo.addPoint(0, 1, 0, lc, 7)
gmsh.model.geo.addPoint(0, 0.8, 0, lc, 8)
gmsh.model.geo.addPoint(0, 0.4, 0, lc, 9)
gmsh.model.geo.addPoint(0.5, 0.4, 0, lc, 10)
gmsh.model.geo.addPoint(0.5, 0.8, 0, lc, 11)

gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 5, 4)
gmsh.model.geo.addLine(5, 6, 5)
gmsh.model.geo.addLine(6, 7, 6)
gmsh.model.geo.addLine(7, 8, 7)
gmsh.model.geo.addLine(8, 9, 8)
gmsh.model.geo.addLine(9, 1, 9)
gmsh.model.geo.addLine(9, 10, 10)
gmsh.model.geo.addLine(10, 3, 11)
gmsh.model.geo.addLine(10, 11, 12)
gmsh.model.geo.addLine(8, 11, 13)
gmsh.model.geo.addLine(11, 4, 14)
gmsh.model.geo.addLine(11, 6, 15)

gmsh.model.geo.addCurveLoop([1, 2, -11, -10, 9], 1)
gmsh.model.geo.addCurveLoop([11, 3, -14, -12], 2)
gmsh.model.geo.addCurveLoop([14, 4, 5, -15], 3)
gmsh.model.geo.addCurveLoop([13, 15, 6, 7], 4)
gmsh.model.geo.addCurveLoop([10, 12, -13, 8], 5)
gmsh.model.geo.addPlaneSurface([1], 1)
gmsh.model.geo.addPlaneSurface([2], 2)
gmsh.model.geo.addPlaneSurface([3], 3)
gmsh.model.geo.addPlaneSurface([4], 4)
gmsh.model.geo.addPlaneSurface([5], 5)

gmsh.model.geo.synchronize()

gmsh.model.addPhysicalGroup(1, [7, 8, 9], 1)
gmsh.model.addPhysicalGroup(1, [2, 3, 4], 2)
gmsh.model.addPhysicalGroup(1, [1], 3)
gmsh.model.addPhysicalGroup(1, [5, 6], 4)
gmsh.model.addPhysicalGroup(1, [10, 11], 5)
gmsh.model.addPhysicalGroup(1, [13, 14], 6)
gmsh.model.addPhysicalGroup(2, [1], 1)
gmsh.model.addPhysicalGroup(2, [2], 2)
gmsh.model.addPhysicalGroup(2, [3], 3)
gmsh.model.addPhysicalGroup(2, [4], 4)
gmsh.model.addPhysicalGroup(2, [5], 5)

# Generate the mesh on rank 0
if comm_world.rank == 0:
    gmsh.model.mesh.generate(2)


In [ ]:
%%px
# Get the cell information from the gmsh model
if comm_world.rank == 0:
    topologies = df.io.gmshio.extract_topology_and_markers(gmsh.model)

    num_cell_types = len(topologies.keys())
    cell_information = dict()
    cell_dimensions = np.zeros(num_cell_types, dtype=np.int32)
    for i, element in enumerate(topologies.keys()):
        _, dim, _, num_nodes, _, _ = gmsh.model.mesh.getElementProperties(element)
        cell_information[i] = {"id": element, "dim": dim, "num_nodes": num_nodes}
        cell_dimensions[i] = dim

    # Sort elements by ascending dimension
    perm_sort = np.argsort(cell_dimensions)
    cell_id = cell_information[perm_sort[-1]]["id"]
    num_nodes = cell_information[perm_sort[-1]]["num_nodes"]
    cell_id, num_nodes = comm_world.bcast([cell_id, num_nodes], root=0)

    cells = np.asarray(topologies[cell_id]["topology"], dtype=np.int64)
    cell_values = [np.asarray(topologies[cell_id]["cell_data"], dtype=np.int32)]
else:
    cell_id, num_nodes = comm_world.bcast([None, None], root=0)
    cells = np.empty([0, num_nodes], dtype=np.int32)
    cell_values = [np.empty((0,), dtype=np.int32)]


In [ ]:
%%px
# choose which region ids to group together (if sufficient ranks are available)
rid_groups = [[1], [2, 5], [3, 4]]

# create a partitioner that partitions the mesh depending on the cell region id
def partitioner(comm, commsize, celltypes, topos):
    nverts = [abs(int(celltype.value)) for celltype in celltypes]
    ncells = sum([int(len(topo)/nverts[t]) for t, topo in enumerate(topos)])
    if commsize == 1:
        # trivial case
        procs = np.zeros(ncells, dtype=np.int32)
        al = df.graph.adjacencylist(procs)
    elif commsize == 2:
        # special case: not a large enough comm so 
        # group the last two rid groups together
        groups = [rid_groups[0], rid_groups[1] + rid_groups[2]]
        procs = np.asarray([g for t in range(len(topos)) for val in cell_values[t] for g, group in enumerate(groups) if val in group], dtype=np.int32)
        al = df.graph.adjacencylist(procs)
    else:
        # arbitrary case
        groups = rid_groups
        if comm.rank == 0:
            # figure out which cells belong together
            cellorder = [None]*ncells
            lgrouptopos = [[[] for topo in topos] for group in groups]
            i = 0
            for g, group in enumerate(groups):
                for t, topo in enumerate(topos):
                    for c, val in enumerate(cell_values[t]):
                        if val in group:
                            lgrouptopos[g][t] += topo[c*nverts[t]:(c+1)*nverts[t]].tolist()
                            cellorder[c] = i # remember the order
                            i += 1
            grouptopos = [[np.array(grouptopo[t], dtype=np.int64) for t in range(len(topos))] for grouptopo in lgrouptopos]
            groupsizes = [max(1, int(sum([len(grouptopo[t])/nverts[t] for t in range(len(topos))])*commsize/ncells)) for grouptopo in grouptopos]
            # correct the logic above so that sum(groupsizes) == commsize
            while sum(groupsizes) < commsize:
                for g in [1, 0, 2]:
                    groupsizes[g] = groupsizes[g] + 1
                    if sum(groupsizes) == commsize: break
            while sum(groupsizes) > commsize:
                for g in [2, 0, 1]:
                    if groupsizes[g] > 1: groupsizes[g] = groupsizes[g] - 1
                    if sum(groupsizes) == commsize: break
            # communicate the groupsizes
            groupsizes = comm.bcast(groupsizes, root=0)
        else:
            grouptopos = [[np.empty((0,), dtype=np.int64) for t in range(len(topos))] for g in range(len(groups))]
            cellorder = []
            groupsizes = comm.bcast([None]*len(groups), root=0)
        if sum(groupsizes) != commsize:
            raise Exception("sum(groupsizes) != commsize")
        values = np.empty((0,), dtype=np.int32)
        # for each group, partition the cells using a cell partitioner
        for g in range(len(groups)):
            lal = df.mesh.create_cell_partitioner(df.mesh.GhostMode.none)(
                comm, groupsizes[g], celltypes, grouptopos[g])
            values = np.concatenate((values, lal.array + sum(groupsizes[:g])), axis=0)
        al = df.graph.adjacencylist(values[cellorder])
    return al

In [ ]:
%%px
# convert the gmsh mesh to a dolfinx mesh
mesh, ct, ft = df.io.gmshio.model_to_mesh(gmsh.model, comm_world, 0, gdim=2, partitioner=partitioner)

In [ ]:
%%px
def create_submesh_split_comm(mesh, cell_indices, cell_tags=None, facet_tags=None):
    """
    Function to return a submesh based on the cell indices provided using a split comm
    that excludes ranks with no cells.

    Arguments:
      * mesh         - original (parent) mesh
      * cell_indices - cell indices of the parent mesh to include in the submesh

    Keyword Arguments:
      * cell_tags    - cell tags on parent mesh that will be mapped and returned relative to submesh (default=None)
      * facet_tags   - facet tags on parent mesh that will be mapped and returned relative to submesh (default=None)

    Returns:
      * submesh            - submesh of mesh given cell_indices
      * submesh_cell_tags  - cell tags relative to submesh if cell_tags provided (otherwise None)
      * submesh_facet_tags - facet tags relative to submesh if facet_tags provided (otherwise None)
      * submesh_cell_map   - map from submesh cells to parent cells
    """
    tdim = mesh.topology.dim
    fdim = tdim-1

    # create the submesh
    submesh, submesh_cell_map, submesh_vertex_map, submesh_geom_map = df.mesh.create_submesh(mesh, tdim, cell_indices)

    # create a split comm
    new_comm = mesh.comm.Split(submesh.topology.index_map(tdim).size_local>0)

    # map from the rank on the original comm to the new split comm
    # used to map index map owners to their new ranks
    # this will include undefined negative numbers on ranks with no corresponding entry on the split comm that should cause failure if the mesh has ghosts on unexpected ranks
    rank_map = np.asarray(submesh.comm.group.Translate_ranks(None, new_comm.group), dtype=np.int32)

    # create a new topology on the new comm
    new_topo = df.cpp.mesh.Topology(new_comm, submesh.topology.cell_type)

    new_topo_im_0 = df.common.IndexMap(new_comm, 
                                    submesh.topology.index_map(0).size_local, 
                                    submesh.topology.index_map(0).ghosts, 
                                    rank_map[submesh.topology.index_map(0).owners])
    new_topo.set_index_map(0, new_topo_im_0)

    new_topo_im_tdim = df.common.IndexMap(new_comm, 
                                    submesh.topology.index_map(tdim).size_local, 
                                    submesh.topology.index_map(tdim).ghosts, 
                                    rank_map[submesh.topology.index_map(tdim).owners])
    new_topo.set_index_map(tdim, new_topo_im_tdim)

    new_topo.set_connectivity(submesh.topology.connectivity(0,0), 0, 0)
    new_topo.set_connectivity(submesh.topology.connectivity(tdim, 0), tdim, 0)

    gdim = submesh.geometry.dim
    new_geom_im = df.common.IndexMap(new_comm,
                                    submesh.geometry.index_map().size_local,
                                    submesh.geometry.index_map().ghosts,
                                    rank_map[submesh.geometry.index_map().owners])
    new_geom = type(submesh.geometry._cpp_object)(new_geom_im, submesh.geometry.dofmap, 
                                    submesh.geometry.cmap._cpp_object, 
                                    submesh.geometry.x[:,:gdim], 
                                    submesh.geometry.input_global_indices)
    
    # set up a new submesh
    new_submesh = df.mesh.Mesh(type(submesh._cpp_object)(new_comm, new_topo, new_geom), submesh.ufl_domain())
    new_submesh.topology.create_connectivity(fdim, tdim)

    # if cell_tags are provided then map to the new submesh
    new_submesh_cell_tags = None
    if cell_tags is not None:
        submesh_cell_tags_indices = []
        submesh_cell_tags_values  = []
        # loop over the submesh cells, checking if they're included in
        # the parent cell_tags
        for i,parentind in enumerate(submesh_cell_map):
            parent_cell_tags_indices = np.argwhere(cell_tags.indices==parentind)
            if parent_cell_tags_indices.shape[0]>0:
                submesh_cell_tags_indices.append(i)
                submesh_cell_tags_values.append(cell_tags.values[parent_cell_tags_indices[0][0]])
        submesh_cell_tags_indices = np.asarray(submesh_cell_tags_indices)
        submesh_cell_tags_values  = np.asarray(submesh_cell_tags_values)

        # create a new meshtags object
        # indices should already be sorted by construction
        new_submesh_cell_tags = df.mesh.meshtags(new_submesh, tdim, 
                                             submesh_cell_tags_indices, 
                                             submesh_cell_tags_values)
            

    # if facet_tags are provided then map to the new submesh
    new_submesh_facet_tags = None
    if facet_tags is not None:
        # parent facet to vertices adjacency list
        f2vs = mesh.topology.connectivity(fdim, 0)

        # submesh facet to vertices adjaceny list
        new_submesh.topology.create_connectivity(fdim, 0)
        submesh_f2vs = new_submesh.topology.connectivity(fdim, 0)
        # create a map from the parent vertices to the submesh facets
        # (only for the facets that exist in the submesh)
        submesh_parentvs2subf = dict()
        for i in range(submesh_f2vs.num_nodes):
            submesh_parentvs2subf[tuple(sorted([submesh_vertex_map[j] for j in submesh_f2vs.links(i)]))] = i

        # loop over the facet_tags and map from the parent facet to the submesh facet
        # via the vertices, copying over the facet_tag values
        submesh_facet_tags_indices = []
        submesh_facet_tags_values  = []
        for i,parentind in enumerate(facet_tags.indices):
            subind = submesh_parentvs2subf.get(tuple(sorted(f2vs.links(parentind))), None)
            if subind is not None:
                submesh_facet_tags_indices.append(subind)
                submesh_facet_tags_values.append(facet_tags.values[i])
        submesh_facet_tags_indices = np.asarray(submesh_facet_tags_indices)
        submesh_facet_tags_values  = np.asarray(submesh_facet_tags_values)

        perm = np.argsort(submesh_facet_tags_indices)
        new_submesh_facet_tags = df.mesh.meshtags(new_submesh, fdim, 
                                              submesh_facet_tags_indices[perm], 
                                              submesh_facet_tags_values[perm])
    
    return new_submesh, new_submesh_cell_tags, new_submesh_facet_tags
    

In [ ]:
%%px
submesh_0, submesh_0_ct, submesh_0_ft = create_submesh_split_comm(mesh, 
                                                    np.concatenate([ct.find(rid) for rid in rid_groups[0]]),
                                                    cell_tags=ct, facet_tags=ft)
submesh_1, submesh_1_ct, submesh_1_ft = create_submesh_split_comm(mesh, 
                                                    np.concatenate([ct.find(rid) for rid in rid_groups[1]]),
                                                    cell_tags=ct, facet_tags=ft)

# Plotting

In [ ]:
%%px
import sys, os
basedir = ''
if "__file__" in globals(): basedir = os.path.dirname(__file__)
sys.path.append(os.path.join(basedir, os.path.pardir, 'python'))
import utils
tdim = mesh.topology.dim

### Whole mesh (cells)

In [ ]:
%%px
plotter_mesh = utils.plot_mesh(mesh, tags=ct, gather=True, show_edges=True, line_width=1)
utils.plot_show(plotter_mesh)

In [ ]:
%%px
plotter_mesh = utils.plot_mesh(mesh, tags=ct, gather=False, show_edges=True, line_width=1, clim=[1,5])
utils.plot_show(plotter_mesh)

### Submesh 0 (cells)

In [ ]:
%%px
if submesh_0.topology.index_map(tdim).size_local>0:
    plotter_mesh = utils.plot_mesh(submesh_0, tags=submesh_0_ct, gather=True, show_edges=True, line_width=1, clim=[1,5])
    utils.plot_show(plotter_mesh)

In [ ]:
%%px
if submesh_0.topology.index_map(tdim).size_local>0:
    plotter_mesh = utils.plot_mesh(submesh_0, tags=submesh_0_ct, gather=False, show_edges=True, line_width=1, clim=[1,5])
    utils.plot_show(plotter_mesh)

### Submesh 1 (cells)

In [ ]:
%%px
if submesh_1.topology.index_map(tdim).size_local>0:
    plotter_mesh = utils.plot_mesh(submesh_1, tags=submesh_1_ct, gather=True, show_edges=True, line_width=1, clim=[1,5])
    utils.plot_show(plotter_mesh)

In [ ]:
%%px
if submesh_1.topology.index_map(tdim).size_local>0:
    plotter_mesh = utils.plot_mesh(submesh_1, tags=submesh_1_ct, gather=False, show_edges=True, line_width=1, clim=[1,5])
    utils.plot_show(plotter_mesh)

### Whole mesh (facets)

In [ ]:
%%px
plotter_mesh = utils.plot_mesh(mesh, tags=ft, gather=True, show_edges=True, line_width=1)
utils.plot_show(plotter_mesh)

In [ ]:
%%px
plotter_mesh = utils.plot_mesh(mesh, tags=ft, gather=False, show_edges=True, line_width=1, clim=[0,6])
utils.plot_show(plotter_mesh)

In [ ]:
%%px
if submesh_0.topology.index_map(tdim).size_local>0:
    plotter_mesh = utils.plot_mesh(submesh_0, tags=submesh_0_ft, gather=True, show_edges=True, line_width=1, clim=[0,6])
    utils.plot_show(plotter_mesh)

In [ ]:
%%px
if submesh_0.topology.index_map(tdim).size_local>0:
    plotter_mesh = utils.plot_mesh(submesh_0, tags=submesh_0_ft, gather=False, show_edges=True, line_width=1, clim=[0,6])
    utils.plot_show(plotter_mesh)

In [ ]:
%%px
if submesh_1.topology.index_map(tdim).size_local>0:
    plotter_mesh = utils.plot_mesh(submesh_1, tags=submesh_1_ft, gather=True, show_edges=True, line_width=1, clim=[0,6])
    utils.plot_show(plotter_mesh)

In [ ]:
%%px
if submesh_1.topology.index_map(tdim).size_local>0:
    plotter_mesh = utils.plot_mesh(submesh_1, tags=submesh_1_ft, gather=False, show_edges=True, line_width=1, clim=[0,6])
    utils.plot_show(plotter_mesh)

In [ ]:
%%px
import ufl
import dolfinx as df


ds_0 = ufl.Measure("ds", domain=submesh_0, subdomain_data=submesh_0_ft)
print(df.fem.assemble_scalar(df.fem.form(df.fem.Constant(submesh_0, df.default_scalar_type(1.0))*ds_0((1,2,3,5,)))))

ds_1 = ufl.Measure("ds", domain=submesh_1, subdomain_data=submesh_1_ft)
print(df.fem.assemble_scalar(df.fem.form(df.fem.Constant(submesh_1, df.default_scalar_type(1.0))*ds_1((1,2,5,6,)))))